In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# probability model
from scipy.stats import multivariate_normal

In [2]:
# Use the generated I3D-based boundaries (var-T, FNS ✗)
nalist = np.load(r"..\list\nalist_i3d.npy")
total_T = int(nalist[-1, 1])

# concat_UCF.npy was created via np.memmap (raw float32 file), so load with memmap
train_data = np.memmap(r"C:\Users\jplabuser\C2FPL\concat_UCF.npy", dtype="float32", mode="r", shape=(total_T, 10, 2048))

print("nalist shape:", nalist.shape, " total_T:", total_T)
print("train_data shape:", train_data.shape)

nalist shape: (1609, 2)  total_T: 779951
train_data shape: (779951, 10, 2048)


In [3]:
# Sanity check: video-0 span in the concatenated memmap
a0, b0 = map(int, nalist[0])
print("video0 span:", (a0, b0), "T0:", b0-a0)

x0 = train_data[a0:b0]
print("slice x0:", x0.shape)        # 여기서 (171,10,2048) 나와야 정상


video0 span: (0, 171) T0: 171
slice x0: (171, 10, 2048)


In [4]:
new_repr = []
for i, (fromid, toid) in enumerate(nalist):
    new_repr.append(train_data[fromid:toid])

len(new_repr)

1609

In [5]:
print("new_repr[0]:", new_repr[0].shape)

new_repr[0]: (171, 10, 2048)


In [6]:
new_repr[i].shape

(151, 10, 2048)

In [7]:
def estimate_gauss(X):
    m = X.shape[0]   # using only first dimension as we know it has only one feature - l2 norm
    
    mu = np.mean(X, axis=0)
    var = np.cov(X.T)
    
    return mu, var

def covariance_mat(X):
    X = np.mean(X , axis= 1)
    X =  X.transpose(1,0)
    cov  = np.cov(X)

    return cov

def get_matrix(data): #입력 (T,10,2048)

    l2_norm = np.sqrt(np.sum(np.square(data), axis=2)) #2048차원에 대해 제곱합 -> 각 crop별 l2norm
    n_train_crop_l2_norm_mean = np.mean(l2_norm, axis= 1)  #crop축 평균 -> 스니펫별 대표 l2norm 값 벡터

    return n_train_crop_l2_norm_mean #(T,)


def diff_l2(new_repr):

    l2_norms = []
    for i in range(len(new_repr)):
        l2_norms.append(get_matrix(new_repr[i]))

    mean_v_l2 = []
    for i in range(len(l2_norms)):
        mean_v_l2.append(np.diff(l2_norms[i], n=1).max())
    return mean_v_l2


In [8]:
params = []
for i in range(len(new_repr)):

    param = get_matrix(new_repr[i])  
    mu, var = estimate_gauss(param)

    params.append((mu, var, )) #(N,2) -> 2차원 특징 벡터(mu, var)로 요약된 비디오 데이터

In [9]:
np.array(params).shape

(1609, 2)

In [10]:
#CPL
from sklearn.mixture import GaussianMixture
import time

gmm = GaussianMixture(n_components=2, max_iter=150, random_state=0, covariance_type='spherical')
# gmm_scores = gmm.score_samples(params)
labels = gmm.fit_predict(params) 

y_gmm = gmm.fit_predict(params)
print(y_gmm.sum(), y_gmm.sum() / len(y_gmm))


score = y_gmm 
score = gmm.score_samples(params)   #log-likelihood
pct_threshold = np.percentile(score, 3) #하위 3%를 outlier로 간주해 res=1(이상) 처리
print(f'The threshold of the score is {pct_threshold:.2f}') 
res = np.array([1 if x < pct_threshold else 0 for x in score]) 
print(res.sum())

233 0.14481044126786824
The threshold of the score is -8.89
49


In [11]:
abnormal_portion = np.where(labels == 1)[0]
normal_portion = np.where(labels == 0)[0]
normal_portion.shape, abnormal_portion.shape

((1376,), (233,))

In [12]:
n_params = np.array(params)[normal_portion]
a_params = np.array(params)[abnormal_portion]
n_params.shape, a_params.shape

((1376, 2), (233, 2))

In [13]:
abag = list(zip(list(np.array(params)[abnormal_portion]), abnormal_portion))
nbag = list(zip(list(np.array(params)[normal_portion]), normal_portion))
len(abag), len(nbag)

(233, 1376)

In [14]:
print("len(nbag):", len(nbag))
print("type(nbag[0]):", type(nbag[0]))

# nbag[0]이 튜플/리스트면 feature가 [0]에 있음
try:
    x0 = nbag[0][0]
    print("type(nbag[0][0]):", type(x0))
    print("shape(nbag[0][0]):", np.array(x0).shape)
except Exception as e:
    print("nbag[0][0] 접근 실패:", e)
    print("shape(nbag[0]):", np.array(nbag[0]).shape)


len(nbag): 1376
type(nbag[0]): <class 'tuple'>
type(nbag[0][0]): <class 'numpy.ndarray'>
shape(nbag[0][0]): (2,)


In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.mixture import GaussianMixture

In [16]:

nu = 1.0
step = 1
import time
start = time.time()
while len(abag) / len(nbag) < nu:
    
    temp_bag = nbag
    X = np.vstack([np.asarray(t[0], dtype=np.float32).reshape(-1) for t in temp_bag])  # (N, 2)
    y_gmm = gmm.fit_predict(X)
    score = gmm.score_samples(X)

    pct_threshold = np.percentile(score, 3) 
    res = np.array([1 if x < pct_threshold else 0 for x in score]) 
    print(f'The threshold of the score in step {step} is {pct_threshold:.2f}, abnormal part: {res.sum()}') 
    
    abnormal_portion = np.where(res == 1)[0]
    normal_portion = np.where(res == 0)[0]
    # abnormal_portion / normal_portion 은 index 배열 (np.where 결과)
    abag += [temp_bag[i] for i in abnormal_portion.tolist()]
    nbag  = [temp_bag[i] for i in normal_portion.tolist()]
    step += 1

print(time.time() - start)

The threshold of the score in step 1 is -6.82, abnormal part: 42
The threshold of the score in step 2 is -6.50, abnormal part: 40
The threshold of the score in step 3 is -6.39, abnormal part: 39
The threshold of the score in step 4 is -6.20, abnormal part: 38
The threshold of the score in step 5 is -6.14, abnormal part: 37
The threshold of the score in step 6 is -5.96, abnormal part: 36
The threshold of the score in step 7 is -5.81, abnormal part: 35
The threshold of the score in step 8 is -5.74, abnormal part: 34
The threshold of the score in step 9 is -5.64, abnormal part: 33
The threshold of the score in step 10 is -5.56, abnormal part: 32
The threshold of the score in step 11 is -5.44, abnormal part: 31
The threshold of the score in step 12 is -5.30, abnormal part: 30
The threshold of the score in step 13 is -5.12, abnormal part: 29
The threshold of the score in step 14 is -4.98, abnormal part: 28
The threshold of the score in step 15 is -4.89, abnormal part: 27
The threshold of th

In [17]:
print(np.where(np.array([x[1] for x in abag]) < 810)[0].shape, len([x[1] for x in abag]))
print('correctness acc: ', np.where(np.array([x[1] for x in abag]) < 810)[0].shape[0] / len([x[1] for x in abag]))

(410,) 821
correctness acc:  0.4993909866017052


In [18]:
print(np.where(np.array([x[1] for x in nbag]) > 810)[0].shape, len([x[1] for x in nbag]))
print('correctness acc: ', np.where(np.array([x[1] for x in nbag]) > 810)[0].shape[0] / len([x[1] for x in nbag]))

(387,) 788
correctness acc:  0.49111675126903553


In [19]:
temp = [k[1] for k in sorted([(x[1], 1.0) for x in abag] + [(x[1], 0.0) for x in nbag], key=lambda z: z[0])]
sum(temp), len(temp)

(821.0, 1609)

### fpl

In [20]:
# normal set creation
normal_set = {}

for i in range(len(new_repr)):
    if temp[i] == 0.0:
        normal_set[i] = new_repr[i]

In [21]:
# abnormal set creation
abnormal_set = {}
for i in range(len(new_repr)):
    if temp[i] == 1.0:
        abnormal_set[i] = new_repr[i]

In [22]:
l2_norms_N = np.empty(0,)
for (idel, sample) in normal_set.items(): 
    
    # print(sample.shape)
    
    #normal 비디오들에서 모든 segment들의 l2 값을 한 벡터로 모음.
    l2_norms_N = np.append(l2_norms_N,get_matrix(sample))

In [23]:
len(normal_set), len(abnormal_set)

(788, 821)

In [24]:
mu_GMM, var_GMM = estimate_gauss(np.array(l2_norms_N))

In [25]:
# probability model
from scipy.stats import multivariate_normal
p = multivariate_normal(mu_GMM, var_GMM)

In [26]:
''' 요게 원래 버전
ground_truth = {} 
length = 0.2 
for (idel, sample) in abnormal_set.items(): 

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    probs = p.pdf(sample_matrix)
    probs.shape
    temp_list = []
    temp_list += [0.0] * len(probs)
    
    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)
    #p value 변화량이 가장 큰 구간을 이상 구간으로
    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth[idel] = temp_list
'''
import math

ground_truth = {}
beta = 0.2

for (idel, sample) in abnormal_set.items():
    sample_matrix = get_matrix(sample)
    '''확인용
    arr = np.asarray(sample_matrix)
    print("idel:", idel)
    print("sample type/shape:", type(sample), getattr(sample, "shape", None))
    print("sample_matrix type:", type(sample_matrix))
    print("sample_matrix np.shape:", arr.shape, "ndim:", arr.ndim, "dtype:", arr.dtype)
    '''
    probs = p.pdf(sample_matrix)
    probs = np.asarray(probs, dtype=np.float64)
    print("probs shape:", probs.shape, "ndim:", probs.ndim)

    m = len(probs)
    if m==0:
        ground_truth[idel] = []
        continue

    window_size = int(math.ceil(beta*m))
    window_size = max(1, min(window_size, m))

    csum = np.cumsum(np.concatenate(([0.0], probs)))
    win_sums = csum[window_size:] - csum[:-window_size]
    win_means = win_sums / window_size

    start = int(np.argmin(win_means))

    conf = -np.log(probs + 1e-12)
    conf = (conf - conf.min()) / (conf.max() - conf.min() + 1e-12)

    '''
    변화량 기준 추가
    delta = np.abs(np.diff(conf, prepend=conf[0]))
    delta = (delta - delta.min()) / (delta.max() - delta.min() + 1e-12)

    conf_final = conf * delta
    '''
    

    temp_list = np.zeros(m,dtype=np.float32)
    temp_list[start:start+window_size] = conf[start:start+window_size]  #soft label로 변경

    ground_truth[idel] = temp_list.tolist()




probs shape: (232,) ndim: 1
probs shape: (1050,) ndim: 1
probs shape: (274,) ndim: 1
probs shape: (63,) ndim: 1
probs shape: (71,) ndim: 1
probs shape: (228,) ndim: 1
probs shape: (321,) ndim: 1
probs shape: (162,) ndim: 1
probs shape: (493,) ndim: 1
probs shape: (88,) ndim: 1
probs shape: (120,) ndim: 1
probs shape: (152,) ndim: 1
probs shape: (247,) ndim: 1
probs shape: (194,) ndim: 1
probs shape: (120,) ndim: 1
probs shape: (177,) ndim: 1
probs shape: (62,) ndim: 1
probs shape: (64,) ndim: 1
probs shape: (274,) ndim: 1
probs shape: (278,) ndim: 1
probs shape: (297,) ndim: 1
probs shape: (177,) ndim: 1
probs shape: (98,) ndim: 1
probs shape: (294,) ndim: 1
probs shape: (113,) ndim: 1
probs shape: (214,) ndim: 1
probs shape: (1780,) ndim: 1
probs shape: (242,) ndim: 1
probs shape: (247,) ndim: 1
probs shape: (318,) ndim: 1
probs shape: (73,) ndim: 1
probs shape: (214,) ndim: 1
probs shape: (134,) ndim: 1
probs shape: (304,) ndim: 1
probs shape: (112,) ndim: 1
probs shape: (191,) ndim:

In [27]:
len(probs)

151

In [28]:
final_gt = []
abnormal_gt = []
for i in range(len(new_repr)):
    if i in normal_set.keys():
        final_gt += [0.0] * new_repr[i].shape[0]
    else:
        final_gt += ground_truth[i]
        abnormal_gt+= ground_truth[i]

In [29]:
len(final_gt)

779951

In [30]:
final_gt = np.array(final_gt, dtype=np.float32)

print("final_gt shape:", final_gt.shape, "sum:", final_gt.sum())
np.save(r"..\Unsup_labels\UCF_unsup_labels_i3d_varT.npy", final_gt)

final_gt shape: (779951,) sum: 19526.697


In [32]:
print("final abag:", len(abag), "nbag:", len(nbag), "ratio:", len(abag)/len(nbag))


final abag: 821 nbag: 788 ratio: 1.0418781725888324


In [33]:
pseudo = np.load("../Unsup_labels/UCF_unsup_labels_i3d_varT.npy")
print(pseudo.shape, pseudo.dtype)
print("mean:", pseudo.mean(), "sum:", pseudo.sum()) #sum 결과 = 전체 스니펫(779951개) 중 약 5.7만 개를 이상(1)으로 찍었음을 알 수 있음. 
print("ones ratio:", pseudo.mean())


(779951,) float32
mean: 0.0250358 sum: 19526.697
ones ratio: 0.0250358


In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_entropy_1.npy', final_gt) #UCF
# np.save('Unsup_labels/'+'XD_I3D_unsup_labels_10_V2_GMM.npy', final_gt) #XD